In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import statsmodels.api as stm
import statsmodels.stats.weightstats as ws
import sklearn.linear_model as sk_lin_mod
import math
import seaborn as sns
import os
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import OLSInfluence
from scipy import stats
from sklearn.model_selection  import train_test_split
from sklearn.linear_model import LinearRegression
import PyQt5 #works
from PyQt5 import QtCore, QtGui, QtWidgets #works for pyqt5
%matplotlib 

## feature selection
### https://www.youtube.com/watch?v=VCVhwjbI6h8
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

Using matplotlib backend: <object object at 0x7f153dc61080>


In [2]:
# os.getcwd()
# print (os.getcwd())
currDir = os.getcwd()
fullPath = currDir + "/CabaneASucrev0r3.csv"
type(currDir)

str

In [3]:
## read csv
donnee = pd.read_csv(fullPath)
stats=donnee.describe()
dimensions=donnee.shape
stats

,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,Production moyenne par entaille (L),...,Pixel6_B,Pixel7_R,Pixel7_G,Pixel7_B,Pixel8_R,Pixel8_G,Pixel8_B,Pixel9_R,Pixel9_G,Pixel9_B
count,1875.000000,1875.000000,1873.00000,1875.000000,1873.000000,1875.000000,1872.000000,1875.000000,1875.000000,1875.000000,...,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000
mean,2010.000000,45.144000,1.51543,-8.633867,-3.557768,10.147680,2.738568,610.334347,20.633067,26.523733,...,33.771435,157.936992,71.317269,32.709104,156.869221,70.605899,33.460053,157.787557,70.609437,32.820418
std,6.059559,25.781811,7.63530,8.695859,7.843854,4.770659,5.574896,126.965290,5.251394,10.234991,...,26.416038,85.971146,52.296358,26.692262,85.231625,52.970868,27.297388,85.760582,53.761268,27.040843
min,2000.000000,1.000000,-22.60000,-34.800000,-25.700000,0.900000,0.000000,313.200000,14.000000,11.567238,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2005.000000,23.000000,-3.60000,-15.100000,-9.300000,6.700000,0.000000,536.400000,16.000000,17.280176,...,8.345000,111.895000,32.555000,4.865000,111.960000,31.290000,5.545000,109.940000,28.830373,4.326591
50%,2010.000000,45.000000,1.70000,-6.700000,-2.500000,9.600000,0.000000,620.300000,20.000000,26.462002,...,32.970000,183.190000,71.920000,31.110000,180.810000,69.960000,31.410000,183.020000,70.558732,31.111611
75%,2015.000000,67.000000,6.40000,-1.450000,2.400000,13.100000,2.600000,685.100000,25.000000,34.315361,...,55.820000,227.420000,102.465000,54.910000,224.955000,103.015000,56.400000,225.820000,104.848430,55.027608
max,2020.000000,90.000000,27.50000,10.000000,18.800000,31.000000,41.600000,811.700000,30.000000,46.784424,...,108.800000,255.000000,255.000000,125.400000,255.000000,255.000000,117.940000,255.000000,255.000000,132.121595


\-----------------\

Pré traitement

\------------------\


In [4]:
donnee.isnull().sum()

Date                                   0
Année                                  0
Jour Calendrier Saison                 0
Temp max.(°C)                          2
Temp min.(°C)                          0
Temp moy.(°C)                          2
Diff Temp (°C)                         0
Précip. tot. (mm)                      3
Précip. Tot. Hiver (mm)                0
Nombre épisodes gel/dégel              0
Production moyenne par entaille (L)    0
Débit sève (L/j)                       0
Sucre sève (%)                         0
Alimentation osmoseur (L/j)            0
Osmoseur (heures opération/j)          0
Pression osmoseur (bar)                0
Sucre sortie osmoseur (%)              0
Température Bouilloire (0C)            0
Temps bouilloire (h)                   0
Sucre du sirop obtenu (%)              0
Quantité de sirop obtenue (L)          0
Transmittance produit (%)              0
Pixel1_R                               0
Pixel1_G                               0
Pixel1_B        

Heureusement, très peu de nulls.

Quels sont les enregistrements contenant des nulls?


In [5]:
donnee[(donnee['Temp max.(°C)'].isna()==True) | (donnee['Précip. tot. (mm)'].isna()==True)] 

,Date,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,...,Pixel7_R,Pixel7_G,Pixel7_B,Pixel8_R,Pixel8_G,Pixel8_B,Pixel9_R,Pixel9_G,Pixel9_B,Classe Sirop
1519,2017-02-02,2017,2,NaN,-17.4,NaN,17.4,NaN,770.6,20,...,224.35,173.08,30.45,255.00,127.83,77.18,255.00,127.017170,10.139073,Ambré
1524,2017-02-07,2017,7,NaN,-25.1,NaN,25.1,NaN,770.6,20,...,255.00,131.77,3.27,225.25,123.51,61.07,194.07,70.795317,71.603721,Foncé
1632,2018-02-26,2018,26,3.2,-7.4,-2.1,10.6,NaN,811.7,16,...,213.40,111.76,3.39,212.26,98.49,35.60,193.85,127.018875,37.339503,Foncé


Fill na

In [6]:
donnee.fillna(method='bfill', inplace=True)
donnee.isnull().sum()

Date                                   0
Année                                  0
Jour Calendrier Saison                 0
Temp max.(°C)                          0
Temp min.(°C)                          0
Temp moy.(°C)                          0
Diff Temp (°C)                         0
Précip. tot. (mm)                      0
Précip. Tot. Hiver (mm)                0
Nombre épisodes gel/dégel              0
Production moyenne par entaille (L)    0
Débit sève (L/j)                       0
Sucre sève (%)                         0
Alimentation osmoseur (L/j)            0
Osmoseur (heures opération/j)          0
Pression osmoseur (bar)                0
Sucre sortie osmoseur (%)              0
Température Bouilloire (0C)            0
Temps bouilloire (h)                   0
Sucre du sirop obtenu (%)              0
Quantité de sirop obtenue (L)          0
Transmittance produit (%)              0
Pixel1_R                               0
Pixel1_G                               0
Pixel1_B        

Pour les question 1 et 2, à fin de faciliter le traitement du detaframe:
Supprimer les colonnes pixel

In [7]:
### obtenir colonne sans pixel
donnee.filter(like='Pixel').columns
dfSansPixel = donnee.drop(donnee.filter(like='Pixel').columns, axis=1)

## enlever aussi Classe Sirop (utiliser dans la question 3)
dfSansPixel = dfSansPixel.drop(dfSansPixel.filter(like='Classe Sirop').columns, axis=1)

dfSansPixel.columns

Index(['Date', 'Année', 'Jour Calendrier Saison', 'Temp max.(°C)',
       'Temp min.(°C)', 'Temp moy.(°C)', 'Diff Temp (°C)', 'Précip. tot. (mm)',
       'Précip. Tot. Hiver (mm)', 'Nombre épisodes gel/dégel',
       'Production moyenne par entaille (L)', 'Débit sève (L/j)',
       'Sucre sève (%)', 'Alimentation osmoseur (L/j)',
       'Osmoseur (heures opération/j)', 'Pression osmoseur (bar)',
       'Sucre sortie osmoseur (%)', 'Température Bouilloire (0C)',
       'Temps bouilloire (h)', 'Sucre du sirop obtenu (%)',
       'Quantité de sirop obtenue (L)', 'Transmittance produit (%)'],
      dtype='object')

In [8]:
dfSansPixel.dtypes

Date                                    object
Année                                    int64
Jour Calendrier Saison                   int64
Temp max.(°C)                          float64
Temp min.(°C)                          float64
Temp moy.(°C)                          float64
Diff Temp (°C)                         float64
Précip. tot. (mm)                      float64
Précip. Tot. Hiver (mm)                float64
Nombre épisodes gel/dégel                int64
Production moyenne par entaille (L)    float64
Débit sève (L/j)                       float64
Sucre sève (%)                         float64
Alimentation osmoseur (L/j)            float64
Osmoseur (heures opération/j)          float64
Pression osmoseur (bar)                float64
Sucre sortie osmoseur (%)              float64
Température Bouilloire (0C)            float64
Temps bouilloire (h)                   float64
Sucre du sirop obtenu (%)                int64
Quantité de sirop obtenue (L)          float64
Transmittance

In [9]:
dfSansPixel.describe().apply(lambda x: round(x, 2))

,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,Production moyenne par entaille (L),...,Sucre sève (%),Alimentation osmoseur (L/j),Osmoseur (heures opération/j),Pression osmoseur (bar),Sucre sortie osmoseur (%),Température Bouilloire (0C),Temps bouilloire (h),Sucre du sirop obtenu (%),Quantité de sirop obtenue (L),Transmittance produit (%)
count,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,...,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00
mean,2010.00,45.14,1.51,-8.63,-3.57,10.15,2.74,610.33,20.63,26.52,...,2.06,1038.45,2.34,34.06,6.84,88.52,2.02,57.02,74.69,37.08
std,6.06,25.78,7.64,8.70,7.84,4.77,5.57,126.97,5.25,10.23,...,0.94,1745.36,3.93,14.39,3.42,37.10,1.72,23.91,126.71,18.92
min,2000.00,1.00,-22.60,-34.80,-25.70,0.90,0.00,313.20,14.00,11.57,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,2005.00,23.00,-3.60,-15.10,-9.30,6.70,0.00,536.40,16.00,17.28,...,2.00,121.00,0.27,37.72,5.72,103.82,1.53,66.00,7.86,29.81
50%,2010.00,45.00,1.70,-6.70,-2.50,9.60,0.00,620.30,20.00,26.46,...,2.34,276.00,0.62,39.60,7.60,104.03,1.62,67.00,19.20,40.70
75%,2015.00,67.00,6.40,-1.45,2.40,13.10,2.60,685.10,25.00,34.32,...,2.67,669.00,1.50,41.12,9.12,104.21,1.69,68.00,44.90,49.60
max,2020.00,90.00,27.50,10.00,18.80,31.00,41.60,811.70,30.00,46.78,...,3.21,10664.62,24.00,46.59,14.59,104.69,11.48,70.00,681.99,82.52


In [10]:
## obtenir un detaframe numérique
dfNumerique = dfSansPixel.select_dtypes(exclude='object').copy()
dfNumerique.describe().apply(lambda x: round(x, 2))

,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,Production moyenne par entaille (L),...,Sucre sève (%),Alimentation osmoseur (L/j),Osmoseur (heures opération/j),Pression osmoseur (bar),Sucre sortie osmoseur (%),Température Bouilloire (0C),Temps bouilloire (h),Sucre du sirop obtenu (%),Quantité de sirop obtenue (L),Transmittance produit (%)
count,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,...,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00
mean,2010.00,45.14,1.51,-8.63,-3.57,10.15,2.74,610.33,20.63,26.52,...,2.06,1038.45,2.34,34.06,6.84,88.52,2.02,57.02,74.69,37.08
std,6.06,25.78,7.64,8.70,7.84,4.77,5.57,126.97,5.25,10.23,...,0.94,1745.36,3.93,14.39,3.42,37.10,1.72,23.91,126.71,18.92
min,2000.00,1.00,-22.60,-34.80,-25.70,0.90,0.00,313.20,14.00,11.57,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,2005.00,23.00,-3.60,-15.10,-9.30,6.70,0.00,536.40,16.00,17.28,...,2.00,121.00,0.27,37.72,5.72,103.82,1.53,66.00,7.86,29.81
50%,2010.00,45.00,1.70,-6.70,-2.50,9.60,0.00,620.30,20.00,26.46,...,2.34,276.00,0.62,39.60,7.60,104.03,1.62,67.00,19.20,40.70
75%,2015.00,67.00,6.40,-1.45,2.40,13.10,2.60,685.10,25.00,34.32,...,2.67,669.00,1.50,41.12,9.12,104.21,1.69,68.00,44.90,49.60
max,2020.00,90.00,27.50,10.00,18.80,31.00,41.60,811.70,30.00,46.78,...,3.21,10664.62,24.00,46.59,14.59,104.69,11.48,70.00,681.99,82.52


In [11]:
### !!! utiliser dans la question3
### convertir Classe Sirop en var numérique et la raoujter au dfNumerique
###
# Creating numeric columns
###
#colsCategSirop = pd.get_dummies(dfSansPixel['Classe Sirop'], prefix='categSirop_', drop_first=True)

## concatenate side by side
#dfNumerique = pd.concat([dfNumerique, colsCategSirop], axis=1)
#dfNumeriue.columns

## Y-at-il des enregistrements ou toutes les variables catégoriques sont 0? 
## Il ne devraient pas en avoir

#dfNumerique[ (dfNumerique['categSirop__Très Foncé']==0) 
#& (dfNumerique['categSirop__Foncé']==0)
#& (dfNumerique['categSirop__Doré']==0)
#& (dfNumerique['categSirop__Ambré']==0)
#].count()

In [12]:
#### enelever les données où débit nul de sève
#dfNumerique = dfNumerique.loc[dfNumerique['Pression osmoseur (bar)'] !=0]
dfNumerique = dfNumerique.loc[dfNumerique['Débit sève (L/j)'] != 0.0]

In [13]:
### fonction pour obtenir un subset d'un dataset contenant seulement les variables indépendantes (x)
def get_x_vars(pDf, pVarY):
    return pDf.drop([pVarY], axis=1)

In [14]:
### retourne un dictionnaire dont key=var dépendante (y) et valeur = dataframe de var x (indépenantes)
def get_dict_y_x(pDf, pListY):
    dict_y_x = {}
    for key in pListY:        
        dict_y_x.update({key: get_x_vars(pDf, key)})
    return dict_y_x

In [15]:
### liste de variables (colonnes) dépendantes
list_cols_dependennt_vars = []
col_debit_seve = dfSansPixel['Débit sève (L/j)'].name
col_sucre_dans_seve = dfSansPixel['Sucre sève (%)'].name
col_pct_transmittance = dfSansPixel['Transmittance produit (%)'].name
col_productivite_seve_par_saison = dfSansPixel['Production moyenne par entaille (L)'].name
list_cols_dependennt_vars = [col_debit_seve, col_sucre_dans_seve, col_pct_transmittance, col_productivite_seve_par_saison]

In [16]:
### dictionnaire contenant les variables dépendantes (y) comme clé et les indépendantes comme df
dict_y_x = get_dict_y_x(dfNumerique, list_cols_dependennt_vars)

In [26]:
dict_y_x['Débit sève (L/j)']

,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,Production moyenne par entaille (L),Sucre sève (%),Alimentation osmoseur (L/j),Osmoseur (heures opération/j),Pression osmoseur (bar),Sucre sortie osmoseur (%),Température Bouilloire (0C),Temps bouilloire (h),Sucre du sirop obtenu (%),Quantité de sirop obtenue (L),Transmittance produit (%)
23,2000,24,5.7,-3.1,1.3,8.8,0.0,313.2,27,34.315361,3.10,4056.0,9.13,40.20,8.20,103.874690,4.27,66,328.89,52.55
24,2000,25,-1.0,-5.3,-3.2,4.3,0.0,313.2,27,34.315361,3.13,179.0,0.40,37.62,5.62,104.050486,1.63,67,15.99,47.29
25,2000,26,4.4,-5.2,-0.4,9.6,0.0,313.2,27,34.315361,2.93,3211.0,7.23,42.76,10.76,104.206423,5.11,68,329.51,37.86
26,2000,27,12.0,2.2,7.1,9.8,1.4,313.2,27,34.315361,2.99,415.0,0.93,41.88,9.88,104.227470,1.65,68,34.30,47.76
27,2000,28,6.6,0.5,3.6,6.1,11.2,313.2,27,34.315361,2.64,268.0,0.60,40.24,8.24,103.834354,1.67,66,19.42,49.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,2020,86,12.5,-2.3,5.1,14.8,0.0,688.3,30,46.784424,3.03,7478.0,16.83,44.60,12.60,103.881904,5.88,66,521.36,46.31
1871,2020,87,10.1,2.9,6.5,7.2,0.0,688.3,30,46.784424,2.90,304.0,0.68,43.15,11.15,104.012573,1.67,67,23.91,38.52
1872,2020,88,12.1,-0.8,5.6,12.9,0.0,688.3,30,46.784424,3.02,6633.0,14.93,39.45,7.45,103.810847,6.03,66,452.75,46.69
1873,2020,89,15.3,-1.5,6.9,16.8,0.0,688.3,30,46.784424,3.00,7067.0,15.90,40.18,8.18,104.641621,8.20,70,552.64,38.02


In [34]:
from featurewiz import featurewiz
for key in dict_y_x:
    print("------------------")
    print("[key]: ", key)
    
    features = featurewiz(dfNumerique, target=key, corr_limit=0.70, 
                                verbose=0) ## verbose = 2 -> generates graph    
    dict_y_x_featurewiz.update({key: dfFeatures}) ## features[1] = dataframe
    

------------------
[key]:  Débit sève (L/j)
############################################################################################
############       F A S T   F E A T U R E  E N G G    A N D    S E L E C T I O N ! ########
# Be judicious with featurewiz. Don't use it to create too many un-interpretable features! #
############################################################################################
Correlation Limit = 0.7
Skipping feature engineering since no feature_engg input...
Skipping category encoding since no category encoders specified in input...
#### Single_Label Regression problem ####
    Loaded train data. Shape = (1595, 21)
    Some column names had special characters which were removed...
#### Single_Label Regression problem ####
No test data filename given...
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
########################

In [ ]:
## book Practical Statiscs for Data Scientists, cahp 2
## https://github.com/gedeck/practical-statistics-for-data-scientists/blob/master/python/notebooks/Chapter%202%20-%20Data%20and%20sampling%20distributions.ipynb
def plot_prob_var_y(pDf):
    for y in list_cols_dependennt_vars:
        col_y = pDf[y][pDf[y]>0]
        print("y", y)
        #print("col[y]", dfNumerique[y][dfNumerique[y]>0])
        np.diff(np.log(col_y))
    
        fig, ax = plt.subplots(figsize=(4, 4))
        ax.set_title( y)
        sts.probplot(col_y, plot=ax)

In [ ]:
#plot_prob_var_y(dfNumerique)

In [ ]:
dfNumerique.describe().apply(lambda x: round(x, 2))

In [ ]:
### Standardization
## https://datagy.io/pandas-normalize-column/
#from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()
#scaler.fit(dfNumerique)
#scaled = scaler.fit_transform(dfNumerique)
#dfNumeriqueStd = pd.DataFrame(scaled, columns=dfNumerique.columns)

In [ ]:
#dfNumeriqueStd.describe().apply(lambda x: round(x, 2))

In [ ]:
def plot_box_plot_vars_y(pDf):
    for y in list_cols_dependennt_vars:
        col_y = pDf[y][pDf[y]>0]
        print("y", y)
    
        plt.figure()
        sns.boxplot(y=col_y).set_title(y)
        plt.show(block=False)

In [ ]:
#plot_box_plot_vars_y(dfNumerique)

### Multiple Linear Regression


https://www.geeksforgeeks.org/ml-multiple-linear-regression-using-python/

Assumption of Regression Model : 

- Linearity: The relationship between dependent and independent variables should be linear.
- Homoscedasticity: Constant variance of the errors should be maintained.
- Multivariate normality: Multiple Regression assumes that the residuals are normally distributed.
- Lack of Multicollinearity: It is assumed that there is little or no multicollinearity in

In [ ]:
#dfNumerique['Temps bouilloire (h)'].sort_values(kind='quicksort')

In [ ]:
#dfNumerique['Temps bouilloire (h)'].describe()

In [ ]:
### col 'Temps bouilloire (h)'
### 4.8 aprox mean + 1.5 std
#outlier = (dfNumerique['Temps bouilloire (h)'].mean() +1.5*dfNumerique['Temps bouilloire (h)'].std())
#dfNumerique['Temps bouilloire (h)'][dfNumerique['Temps bouilloire (h)']>outlier]

In [ ]:
#dfNumerique = dfNumerique.loc[dfNumerique['Temps bouilloire (h)']<outlier]
#dfNumerique.describe()

In [ ]:
#### col osmoseur heures opérations/j
#dfNumerique['Osmoseur (heures opération/j)'].describe()

In [ ]:
#outlier = (dfNumerique['Osmoseur (heures opération/j)'].mean() +1.5*dfNumerique['Osmoseur (heures opération/j)'].std())
#print(outlier)
#dfNumerique['Osmoseur (heures opération/j)'][dfNumerique['Osmoseur (heures opération/j)']]>outlier]

In [ ]:
#dfNumerique = dfNumerique.loc[dfNumerique['Osmoseur (heures opération/j)']<outlier]


In [ ]:
#dfNumerique.describe()

In [ ]:
corr = dfNumerique.corr()

#sorted_pairs = corr.unstack().sort_values(kind="quicksort", ascending=False).dropna(inplace=True)
#corr.unstack().sort_values(kind="quicksort", ascending=False).dropna(inplace=True)
x = corr.unstack().sort_values(kind="quicksort", ascending=False).apply(lambda  x: str(x).split())
type(x)
## reset column names
x = pd.DataFrame(x).reset_index()
x.columns = ['col1', 'col2', 'corr']
## transform corr from [1] to 1
x['corr'] = x['corr'].apply(lambda x: float(np.squeeze(x)))

## paires de colonnes avec des corrélation élevées (> 0.6)
x[(x['corr']>0.6) & (x['col1']!=x['col2'])].sort_values('col1')

In [ ]:
#sns.heatmap(corr, cbar=True, center=0, annot=True, fmt="0.2f", cmap="Blues")

### Séparation des variables dépendantes (Y) et indépendantes (X)

In [ ]:
## seulement les variables X
dfNumerique_X_cols = dfNumerique.loc[:, ~dfNumerique.columns.isin(list_cols_dependennt_vars)].copy()

lstColTemp = ['Temp max.(°C)', 'Temp min.(°C)']
dfNumerique_X_cols = dfNumerique_X_cols.loc[:,~dfNumerique_X_cols.columns.isin(lstColTemp) ].copy()

## seulement les variables Y
dfNumerique_Y_cols = dfNumerique.loc[:, dfNumerique.columns.isin(list_cols_dependennt_vars)].copy()

len(dfNumerique.columns)

In [ ]:
#### Après coup, on a appris que ces 2 variables causaient un prob de haute colinéarité.
#### et si on les enlève plus tôt?

lstColsExlude = ['Précip. Tot. Hiver (mm)', 'Temp moy.(°C)']
dfNumerique_X_cols = dfNumerique_X_cols.loc[:, ~dfNumerique_X_cols.columns.isin([col for col in lstColsExlude])]

#### Evaluate model

=============== Evaluating model ===========

https://www.youtube.com/watch?v=VCVhwjbI6h8

========

In [ ]:
### Fonction qui imprime le sommaire du modèle
def print_model_summary(pDfColsX, pDfColsY):
    ## Add constant
    pDfColsX = stm.add_constant(pDfColsX)
    lst_models = []
    for y in (pDfColsY.columns.values):
    
        col_y = dfNumerique_Y_cols[y]
    
        model = stm.OLS(col_y, pDfColsX.assign(const=1)).fit()
        print("=========")
        print("var depend: ", y)
    
        #lst_models.append(model)
        print(model.summary())
        #sns.distplot(model.resid, fit=sts.norm)        
        #p = sns.histplot(model.resid, kde=True, stat="density").set(title = y)
        

In [ ]:
print("---- Sommaire dfNumerique ----")
print_model_summary(dfNumerique_X_cols, dfNumerique_Y_cols)

In [ ]:
def analyze_model(pColsY, pDfDependentVars):
    for y in (pColsY.columns.values):    
        col_y = pColsY[y]
        X_train, X_test, y_train, y_test = \
        train_test_split(pDfDependentVars, col_y, test_size=0.25, random_state=0)


        ## transforming data    
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.fit_transform(X_test)


        ## Fitting Multiple Linear Regression to the training set
        regressor =  LinearRegression()
        regressor.fit(X_train, y_train)
        

        y_pred = regressor.predict(X_test)
        mse = math.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        #r_sq = regressor.score(X_test, y_test)

        ##adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
        ## Where n is the sample size and p is the number of independent variables.

        #print ("len(X_test): ", len(X_test))
        #print ("len(y_test): ", len(y_test))
    
        ### ajd_r2
        ## https://stackoverflow.com/questions/51038820/how-to-calculated-the-adjusted-r2-value-using-scikit
        ## https://www.dummies.com/article/business-careers-money/business/accounting/calculation-analysis/how-to-calculate-the-adjusted-coefficient-of-determination-146054/
        #print ("n: ", n)
        #print ("p: ", p)
        #print ("adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)")

        n = len(X_test)
        p = len(pDfDependentVars.columns)
        adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    
        print("--------")
        
        print("[col_y]: ", col_y.name)
        print("[mse]: ", mse)
        #print("[r2] - A value of 1 indicates that the response variable can be perfectly explained by the predictor variables.")
        print("[r2]: ", r2)
        print("[adj_r2]: ", adj_r2)
        print("[coef]: ", regressor.coef_)
        print("[intercept_]: ", regressor.intercept_)
        #print("[r_sq]: ", r_sq)

In [ ]:
analyze_model(dfNumerique_Y_cols, dfNumerique_X_cols)

\---

Sans rien faire, on a un modèle très précis (r2 et adj_r2 élevés).
Mais pourrait-on avoir un bon résultat avec moins de colonnes?

\---

In [ ]:
### https://github.com/simaria22/prediction_heart_failure/blob/master/feature_select.py
## Ajoute des colonnes au modèle pendant que la nouvelle colonne améliore
## le modèle OU
## jusqu'à qu'un certain niveau de précision soit atteint
def get_reduced_df_X(pDfx, pY, pAccuracy=0.7):
    print("------")
    print("dependent var y: ", pY.name)
    
    # new X df with fewer columns than pDfx
    reducedDf = []

    # column names to keep
    dfKeeper = None

    #no of features
    nof_list=np.arange(1,len(pDfx.columns))            
    high_score=0
    #Variable to store the optimum features
    nof=0           
    score_list =[]
    for n in range(len(nof_list)):
        X_train, X_test, y_train, y_test = train_test_split(pDfx, pY, test_size = 0.3, random_state = 0)
        model = LinearRegression()
        rfe = RFE(model,n_features_to_select= nof_list[n])
        X_train_rfe = rfe.fit_transform(X_train,y_train)
        X_test_rfe = rfe.transform(X_test)
        model.fit(X_train_rfe,y_train)
        score = model.score(X_test_rfe,y_test)
        score_list.append(score)
        
        print("n: ", n)
        print("score: ", score)
        print("high_score: ", high_score)

        # print summaries for the selection of attributes
        print("rfe.support_: ",rfe.support_)
        print("rfe.ranking_: " ,rfe.ranking_)
        
        #dfKeeper = pd.DataFrame(rfe.support_,index=pDfx.columns,columns=['Rank'])
        data= {'colName': pDfx.columns, 'keep': rfe.support_ }
        dfKeeper = pd.DataFrame(data)
        dfKeeper = dfKeeper[dfKeeper["keep"]==True]
        if(score>high_score):
            high_score = score
            nof = nof_list[n]
        if (score > pAccuracy):
            break

    
    print("Optimum number of features: %d" %nof)
    print("Score with %d features: %f" % (nof, high_score))
    print("df columns to keep: ")
    #print("shape")
    #print(dfNumerique_X_cols[dfKeeper["colName"]].shape)
    print(dfKeeper)
    
    reducedDf = dfNumerique_X_cols[dfKeeper["colName"]]
    
    #print("reducedDf: ", reducedDf.head())
    #print("unique: ", reducedDf[dfKeeper["colName"]].unique())
    return reducedDf

In [ ]:
### création d'un dictionnaire contenant la variable dépendantes et ses variables indépendantes
### correspondantes
### clé = colonne dépendante, valeur = dataframe des colonnes avec haute influence
dict_y_colsX = {}

for y in (dfNumerique_Y_cols.columns.values):    
    col_y = dfNumerique_Y_cols[y]
    reducedDf = get_reduced_df_X(dfNumerique_X_cols, col_y)
    #print("reduceDf.columns.values: ",  reducedDf.columns.values)
    #print("unique values: ", reducedDf[reducedDf.columns.values[0]].unique())
    #print("len: ", len(reducedDf[reducedDf.columns.values[0]]))
    #print("Nan: ", reducedDf[reducedDf.columns.values[0]].isnull().sum())
    ## call to analyze_model
    #analyze_model(dfNumerique_Y_cols, reducedDf)
    #analyze_model(col_y.to_frame(), reducedDf)
    dict_y_colsX.update({col_y.name: reducedDf})

In [ ]:
### analyse le modèle de nouveau avec un dataframe réduit
#analyze_model(dfNumerique_Y_cols, dfNumerique_X_cols)
for key in (dict_y_colsX.keys()):
    print("key: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]
    #print(type(dfX))
    #print(type(dfY))
    #dfX =pd.DataFrame(dict_y_colsX[key], columns=[key])
    #print("shape: ", dfX.shape)
    #print("nulls: ", dfX.isnull().sum())
    analyze_model(dfY.to_frame(), dfX)
    

In [ ]:
### analyse le modèle de nouveau avec un dataframe réduit
#analyze_model(dfNumerique_Y_cols, dfNumerique_X_cols)
for key in (dict_y_colsX.keys()):
    print("key: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]    
    print_model_summary(dfX, dfY.to_frame())

-----

À ce point, on a un modèle avec une bonne précision (R-squared et Adj R-squared) pour 
toutes les variables dépendantes.

Seuelement la variable 'Transmittance produit (%)' contient un advertissement de colinéarité

-----

In [ ]:
#### outliers
### book: Practical statistcs for data scientists, pg 178
## https://github.com/gedeck/practical-statistics-for-data-scientists
dict_outliers = {}
for key in (dict_y_colsX.keys()):
    print("--------------------------")
    print("key: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]    
    outlier = stm.OLS(dfY, dfX.assign(const=1))
    result = outlier.fit()
    influence = OLSInfluence(result)
    sresiduals = influence.resid_studentized_internal
    sresiduals.idxmin(), sresiduals.min()
    outlier = dfNumerique.loc[sresiduals.idxmin(), :]
    print("[outler[key]: ", outlier[key])
    print("[outlier values]: ", outlier[dfX.columns.values])

In [ ]:
### pairplots
#analyze_model(dfNumerique_Y_cols, dfNumerique_X_cols)
i=0
for key in (dict_y_colsX.keys()):
    i+=1
    print("key: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]    
    #sns.pairplot(pd.concat([dfX, dfY], axis=0))
    #plt.figure(i)
    #print(dfX.columns.values)
    #print(dfNumerique[key].head())
    #plt.figure()
    #plt.plot(pd.concat([dfX, dfY], axis=0))
    #####sns.pairplot(data = dfNumerique, x_vars = dfX.columns.values , y_vars = [key], kind='reg', diag_kind=None)
    
    #sns.pairplot(data = reducedDf, x_vars = ['Nombre épisodes gel/dégel'], y_vars = ['Production moyenne par entaille (L)'], kind='reg', diag_kind='kde')
    #plt.show(block=False)

    

In [ ]:
### plot regressions
#https://seaborn.pydata.org/tutorial/regression.html
i=0
for key in (dict_y_colsX.keys()):
    
    print("key: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]    
    for col_x in dfX.columns.values:
        i+=1
        plt.figure(i)
        print("col_x: ", col_x)
        #sns.regplot(data = dfNumerique, x = col_x, y = key)    
        plt.show(block=False)

In [ ]:
i=0
for key in (dict_y_colsX.keys()):
    
    print("key: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]    
    for col_x in dfX.columns.values:
        i+=1
        plt.figure(i)
        print("col_x: ", col_x)
        #sns.boxplot(x=dfX[col_x])  
        #sns.histplot(data=dfX, bins=10, x=col_x)
        plt.show(block=False)

#i=0    
#for col_x in dfX.columns:
#    i+=1
#    plt.figure(i)
    #print("col_x: ", col_x)
    #print(type(col_x))
 #   sns.boxplot(x=col_x.to_frame())    
 #   plt.show(block=False)

In [ ]:
### boxplot vars y
#https://seaborn.pydata.org/tutorial/regression.html
i=0
for key in (dict_y_colsX.keys()):    
    print("key: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]    
    i+=1
    plt.figure(i)    
    #sns.histplot(data=dfY, bins=10)
    plt.show(block=False)

------

la var dépendante Transmittance produit (%) a un problème de multicolinearité 

------

In [ ]:
#### END Evaluate model

In [ ]:
##### Vif - régler la colinéarité de la variable

In [ ]:
### vif
## https://www.kdnuggets.com/2019/07/check-quality-regression-model-python.html
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [ ]:
### https://www.geeksforgeeks.org/detecting-multicollinearity-with-vif-python/

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = dfNumerique_X_cols.columns


In [ ]:

# calculating VIF for each feature

vif_data["VIF"] = [vif(dfNumerique_X_cols.values, i)
                          for i in range(len(dfNumerique_X_cols.columns))]

vif_data.sort_values(by="VIF", axis=0, kind="quicksort",  ascending=False)


In [ ]:
dfVifPlusGrand10 = vif_data[vif_data["VIF"] >10]
len(dfVifPlusGrand10)

In [ ]:
### dfNumerique_X sans les vif >10
dfNumerique_X_cols_vif =[]
dfNumerique_X_cols_vif = dfNumerique_X_cols.loc[:, ~dfNumerique_X_cols.columns.isin(dfVifPlusGrand10["feature"].values)].copy()
#for x in (dfVifPlusGrand10["feature"].values):
#    print("col: ", x)
dfNumerique_X_cols_vif.columns

In [ ]:
sns.heatmap(dfNumerique_X_cols_vif.corr(), cbar=True, cmap="Blues", center=0, annot=True, fmt=".2f")

\-------------------\
Est-ce que l'on obtien un résultat meilleur pour 
'Transmittance produit (%)' ?\
\----------------\

In [ ]:
### appelle get_reduced_df_X avec 
### X =  dfNumerique_X_cols_vif et Y = dfNumerique_Y_cols['Transmittance produit (%)']
newDfTransmittance = get_reduced_df_X(dfNumerique_X_cols_vif, dfNumerique_Y_cols['Transmittance produit (%)'])
print_model_summary(newDfTransmittance, dfNumerique_Y_cols['Transmittance produit (%)'].to_frame())

In [ ]:
analyze_model( dfNumerique_Y_cols['Transmittance produit (%)'].to_frame(), newDfTransmittance)

In [ ]:
### Le résultat n'est pas meilleur
## éliminons 'Précip. Tot. Hiver (mm)' et Temp moy.(°C) à cause de leur haute correlation
## ('Précip. Tot. Hiver (mm)', 'Année') = 0.78 et ('Temp moy.(°C)', 'Calendrier saison') = 0.73


### appelle get_reduced_df_X avec 
### X =  dfNumerique_X_cols_vif et Y = dfNumerique_Y_cols['Transmittance produit (%)']
#dfDependantVars= dfNumerique.loc[:, ~dfNumerique.columns.isin([col.name for col in list_cols_dependennt_vars])]
lstColsExlude = ['Précip. Tot. Hiver (mm)', 'Temp moy.(°C)']
dfNumerique_X_cols_vif = dfNumerique_X_cols_vif.loc[:, ~dfNumerique_X_cols_vif.columns.isin([col for col in lstColsExlude])]
newDfTransmittance = get_reduced_df_X(dfNumerique_X_cols_vif, dfNumerique_Y_cols['Transmittance produit (%)'])
print_model_summary(newDfTransmittance, dfNumerique_Y_cols['Transmittance produit (%)'].to_frame())

In [ ]:
####
###  Fnalement on n'a plus le problème de colinéarité
###

In [ ]:
#### Réponses aux questions 1
### Les tableaux ci-bas donnent, entre autres les informations concernant le adj. r-squared
###, t-value, p-value
### et coefficients des modèles.
### !!!NOTE!!! générer les graphiques et le sauvegarder à part!!!


-----------------------------

#######

\---------------

Q2)
Hypothesis testing

\------------

\--------------\

Vars:
'Transmittance produit (%)'
'Sucre sève (%)'


Y-at-il une variance significative entre les années 2014-2016?


Obtenir la moyenne pour les année 2014 à 2016 e comparer les résultats



\---------------\

In [ ]:
### obtenir moyenne d'une colonne
def getMeanListExclusions(pColName, pData=dfNumerique, pListeExclusion=[2014, 2015, 2016]):
    return pData[pColName][~pData['Année'].isin(pListeExclusion)].mean()

In [ ]:
#moyenne_transmit_toutes_annees = dfNumerique['Transmittance produit (%)'].mean()
moyenne_transmit_toutes_annees = getMeanListExclusions('Transmittance produit (%)')
moyenne_sucre_toutes_annees = getMeanListExclusions('Sucre sève (%)')

In [ ]:
moyenne_transmit_toutes_annees

In [ ]:
moyenne_sucre_toutes_annees

\----------------\

Quelles sont les moyennes pour les années de 2014 à 2016?

\----------------\

In [ ]:
### obtenir moyenne d'une colonne
def getMeanListInclusions(pColName, pAnnee, pData=dfNumerique ):
    return pData[pColName][pData['Année'] == pAnnee].mean()

In [ ]:
### for these dictionnaries, key = year [2014-2016]
lstAnnees = [2014, 2015, 2016]
dictAnneeTransmittance = {}
dictAnneeSucre = {}
for annee in lstAnnees:    
    dictAnneeTransmittance.update({annee: getMeanListInclusions('Transmittance produit (%)', annee)})
    dictAnneeSucre.update({annee: getMeanListInclusions('Sucre sève (%)', annee)})

In [ ]:
dictAnneeTransmittance

In [ ]:
dictAnneeSucre

In [ ]:
### Imnprime la différence entre la moyenne historique (excluant 2014-2016) versus [2014-2016]
def compare_means(pMean1, pMean2, pAnnee, pCol):
    diff = pMean1 - pMean2
    print("-----")
    print("Différence entre moyennes pour ", pCol)
    print("moyenne historique: ", pMean1)
    print("moyenne ", pAnnee)
    print("Différence absolue: ", abs(diff))
    print("Différence %: ", abs(100*(diff/pMean1)))
    

In [ ]:
### Transmittance
for key in dictAnneeTransmittance.keys():
    compare_means(moyenne_transmit_toutes_annees, dictAnneeTransmittance.get(key), key, 'Transmittance produit (%)')

In [ ]:
### Transmittance
for key in dictAnneeSucre.keys():
    compare_means(moyenne_sucre_toutes_annees, dictAnneeSucre.get(key), key, 'Sucre sève (%)')

In [ ]:
## https://towardsdatascience.com/demystifying-hypothesis-testing-with-simple-python-examples-4997ad3c5294
def print_analyse_hypothese(pColYName, pMoyenneHistoriqueY, pDict, pDf=dfNumerique, pConf=0.05):
    ## Transmittance produit (%)
    n =len(pDf)
    pnull = pMoyenneHistoriqueY ##0.43732148760330574 #moyenne hystorique
    ### Transmittance
    print("==================================")
    print("var [pColYName]: ", pColYName)
    for key in pDict.keys():
        phat = pDict.get(key) #moyenne de l'année
        print("-------------------")
        print("année = ", key)
        print("n = ", n)
        print("moyenne historique [pnull] = ", pnull)
        print("moyenne de l'année [phat] = ", phat)
        #sm.stats.proportions_ztest(phat * n, n, pnull, alternative='larger')
        zstat, p_value = stm.stats.ztest(pDf[pColYName][pDf['Année']==key], value = pnull ,alternative='two-sided')        
        #zstat, p_value = stm.stats.proportions_ztest( nobs=n, pnull,  alternative='two-sided')
        print("zstat: ", zstat)
        print("p_value: ", p_value)
        print("h0: moyenne historique = moyenne [2014-2016]")
        print("h1: moyenne historique != moyenne [2014-2016]")
        if (p_value < pConf):
            print("p_value < confidence --> on REJÈTE l'hypothèse null ")
        else:
            print("p_value > confidence --> on accèpete l'hypothèse null")

In [ ]:
print_analyse_hypothese('Transmittance produit (%)', moyenne_transmit_toutes_annees, dictAnneeTransmittance)
print_analyse_hypothese('Sucre sève (%)', moyenne_sucre_toutes_annees, dictAnneeSucre)

In [ ]:
### obtenir colonne  pixel
donnee.filter(like='Pixel').columns
dfPixel = donnee.drop(donnee.filter(like='Pixel').columns, axis=1)

## enlever aussi Classe Sirop (utiliser dans la question 3)
dfPixel = dfSansPixel.drop(dfSansPixel.filter(like='Classe Sirop').columns, axis=1)

dfPixel = donnee.filter(like='Pixel', axis=1)
dfPixel
dfPixel.columns

In [ ]:
## https://towardsdatascience.com/demystifying-hypothesis-testing-with-simple-python-examples-4997ad3c5294
## Transmittance produit (%)
n =len(dfNumerique)
pnull = moyenne_transmit_toutes_annees ##0.43732148760330574 #moyenne hystorique
### Transmittance
confidence = 0.05
for key in dictAnneeTransmittance.keys():
    phat = dictAnneeTransmittance.get(key)
    print("année = ", key)
    print("n = ", n)
    print("pnull = ", pnull)
    print("phat = ", phat)
    #sm.stats.proportions_ztest(phat * n, n, pnull, alternative='larger')
    zstat, p_value = stm.stats.ztest(dfNumerique['Transmittance produit (%)'][dfNumerique['Année']==key], value = pnull ,alternative='two-sided')
    #zstat, p_value = stm.stats.proportions_ztest(44.41 * n, n, 44.036, alternative='larger')
    #zstat, p_value = stm.stats.proportions_ztest( nobs=n, pnull,  alternative='two-sided')
    print("zstat: ", zstat)
    print("p_value: ", p_value)
    if (p_value < confidence):
        print("p_value < confidence --> on rejète l'hypothèse null ")
    else:
        print("p_value > confidence --> on accèpete l'hypothèse null")
    


In [ ]:
#dataframe[dataframe['Percentage'] > 70]
dfNumerique[dfNumerique['Année'] == 2014]
#dfNumerique[dfNumerique['année']>2014]

In [ ]:
#import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
n = 1018
pnull = 52
phat = 56
stm.stats.proportions_ztest(phat * n, n, pnull, alternative='larger')

In [ ]:

pColName = 'Année'
pListeExclusion = [2014, 2015, 2016]
pData = dfNumerique
pData[pColName][~pData[pColName].isin(pListeExclusion)].unique()

In [ ]:
### moyenne transmittance

### boxplot vars y
#https://seaborn.pydata.org/tutorial/regression.html
i=0
for key in (dict_y_colsX.keys()):    
    print("key: ", key)
    #dfX = dict_y_colsX.get(key)
    #dfY = dfNumerique_Y_cols[key]    
    #i+=1
    #plt.figure(i)    
    #sns.histplot(data=dfY, bins=10)
    #plt.show(block=False)

In [ ]:
dfNumerique['Année'].unique()

In [ ]:
sns.scatterplot(data = dfNumerique
                , x = dfNumerique['Année']
                , y = dfNumerique['Transmittance produit (%)']
           )

In [ ]:
sns.relplot(data = dfNumerique, x="Année", y="Transmittance produit (%)", hue="Année")

In [ ]:
## boxenplot
def plotBoxenplot(pColNameY, pColNameX = "Année", pData = dfNumerique):
    i = random.randint(100)
    plt.figure(i)
    sns.boxenplot(data = pData, x=pColNameX, y=pColNameY)
    plt.show(block=False)

In [ ]:
sns.boxenplot(data = dfNumerique, x="Année", y="Transmittance produit (%)")

In [ ]:
sns.boxenplot(data = dfNumerique, x="Année", y="Sucre sève (%)")

In [ ]:
### Q2
### plot kde graph
def plotKde(pColNameX, pHue='Année', pData = dfNumerique):
    plt.figure() 
    sns.kdeplot(data = pData, x=pColNameX, hue=pHue)
    plt.show(block=False)

In [ ]:
###kdeplot
#sns.kdeplot(data = dfNumerique[dfNumerique['Année']>2012], x="Transmittance produit (%)", hue="Année")
#sns.kdeplot(data = dfNumerique, x="Transmittance produit (%)", hue="Année")
plotKde(pColNameX = "Transmittance produit (%)")

In [ ]:
plotKde(pColNameX = "Sucre sève (%)")

\------------\
En principe, un examen visuel ne montre pas une différence "significative"
ni de la transmittance ni du sucre
\--------------\

In [ ]:
dfNumerique.columns.values

##### feature selection ####

https://machinelearningmastery.com/feature-selection-for-regression-data/


In [ ]:
# example of correlation feature selection for numerical data
# compare different numbers of features selected using mutual information
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV

In [ ]:

# define dataset
X = dfNumerique_X_cols.copy()

y = dfNumerique['Débit sève (L/j)']

# define the evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)


# define the pipeline to evaluate
model = LinearRegression()
fs = SelectKBest(score_func=mutual_info_regression)
pipeline = Pipeline(steps=[('sel',fs), ('lr', model)])

# define the grid
grid = dict()
grid['sel__k'] = [i for i in range(X.shape[1]-20, X.shape[1]+1)]



In [ ]:
### https://www.youtube.com/watch?v=VCVhwjbI6h8
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(dfNumerique_X_cols_vif, dfNumerique_Y_cols, test_size=0.25, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(dfNumerique_X_cols_vif, dfNumerique_Y_cols['Production moyenne par entaille (L)'], test_size=0.25, random_state=0)

In [ ]:
## transforming data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


In [ ]:
## Fitting Multiple Linear Regression to the training set
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

regressor =  LinearRegression()
regressor.fit(X_train, y_train)


In [ ]:
y_pred = regressor.predict(X_test)
math.sqrt(mean_squared_error(y_test, y_pred))
r2_score(y_test, y_pred)

In [ ]:
for m in lst_models:
    print("==== SUMMARY ====")
    m.summary()


In [ ]:
#dfNumerique_X_cols_vif.head()

In [ ]:
duncan_prestige = stm.datasets.get_rdataset("Duncan", "carData")
Y = duncan_prestige.data['income']
X = duncan_prestige.data['education']
X = stm.add_constant(X)
print("type(Y)", type(Y))
print("type(X)", type(X))
print("len(Y): ", len(Y))
print("len(X): ", len(X))

In [ ]:
#print(Y.shape)
X.head()
#Y.head()

In [ ]:

model = stm.OLS(Y,X)
results = model.fit()
results.params

###### 

In [ ]:
## On voit qu'il y a certaines correlations parfaites
## ex: (temp moyen, temp max), (temp moyen, temp min), (moyenne entaille, episode gel/degel)
## (pression osmoseur bar, boullioire 0c), (pression osmoseur bar, quantité sirop obtenu %)
## (osmoseur heures opération, alimentation osmoseur (L/j))
## (Sucre sortie osmoseur (%), pression osmoseur)
## (alimentation osmoseur, (L/j), temps boulloire) -> 0.89
## (débit sève, heures opération/j)
## Enlevons quelques unes de ces variables et revoyons la correlation
list_col_redondantes = ['Temp min.(°C)', 'Temp max.(°C)', 'Température Bouilloire (0C)'
                       , 'Quantité de sirop obtenue (L)', 'Sucre du sirop obtenu (%)'
                       , 'Osmoseur (heures opération/j)', 'Sucre sortie osmoseur (%)'
                       , 'Temps bouilloire (h)'
                       ]

## On repète l'opération antérieur avec moins de colonnes
dfNumerique = dfNumerique.loc[:, ~dfNumerique.columns.isin(list_col_redondantes)]

corr = dfNumerique.corr()

## heatmap sans correlations parfaites ou presque parfaite:
sns.heatmap(corr, cbar=True, cmap="Blues", center=0, annot=True, fmt=".2f")

In [ ]:
### pandas.series condition https://www.youtube.com/watch?v=BgfvF6mu20c
### pour chaque var Y, imprimer les variables les plus correlées
### on applique la condition 
for col in list_cols_dependennt_vars:
    print("===========================")
    print(" y = ", col.name)
    cond_correl_plus_grand_50 = ((corr[col.name] > 0.5) & (corr[col.name] <1)) 
    print(corr[col.name][cond_correl_plus_grand_50].sort_values(ascending=False))
    
#list_cols_dependennt_vars

In [ ]:
sns.pairplot(dfNumerique)

### At this point, we have strong relationships:
- Débit sève (L/j) --> Alimentation osmoseur (L/j) [0.97]
- Sucre sève (%) --> Pression osmoseur (bar) [0.9]
- Transmittance produit (%) --> Pression osmoseur (bar)[0.81], Sucre sortie osmoseur (%) [0.68]
- Production moyenne par entaille (L) --> Nombre épisodes gel/dégel [0.95]

In [ ]:
## Transmitance
### pression_osmoseur_vs_transmittance
sns.lmplot(x='Pression osmoseur (bar)', y = 'Transmittance produit (%)', data = dfNumerique)

In [ ]:
## Le graphique ression_osmoseur_vs_transmittance montre une concentration
## dans x près de 40. Regardons s'il y a des outliers:
sns.boxplot(x='Pression osmoseur (bar)',data = dfNumerique)

In [ ]:
## effectivement, le boxplot nous montre la présence des outliers
stats = dfNumerique['Pression osmoseur (bar)'].describe()
stats

In [ ]:
sns.lmplot(x='Alimentation osmoseur (L/j)' , y='Débit sève (L/j)', data=dfNumerique)

In [ ]:
### regardons combien de 0:
sns.histplot(x='Pression osmoseur (bar)',data = dfNumerique)

In [ ]:
#sns.pairplot(dfNumerique)

In [ ]:
## correlation seulement entre les variables indépendantes

dfDependantVars= dfNumerique.loc[:, ~dfNumerique.columns.isin([col.name for col in list_cols_dependennt_vars])]
type(dfDependantVars)

corr2 = dfDependantVars.corr()
sns.heatmap(corr2, cbar=True, annot=True, cmap="Blues", fmt=".02f", center=0)

Multiple Linear Regression - https://www.youtube.com/watch?v=J_LnPL3Qg70


In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()

## https://www.statology.org/pandas-exclude-column/
#select all columns except 'rebounds' and 'assists'
#df.loc[:, ~df.columns.isin(['rebounds', 'assists'])]
list_reg = []
for col in list_cols_dependennt_vars:
 #   print(col.name)
    #result = reg.fit(dfNumerique[:, ~dfNumerique.columns.isin([col.name for col in list_cols_dependennt_vars])], dfNumerique[col.name])
    result = reg.fit(dfNumerique.loc[:, ~dfNumerique.columns.isin([col.name for col in list_cols_dependennt_vars])], dfNumerique[col.name])
    list_reg.append(result)


#dfNumerique['Année'].name
#list_cols_dependennt_vars[0].name
#[col.name for col in list_cols_dependennt_vars]
#reg.fit(dfNumerique.loc[:, ~dfNumerique.columns.isin([col.name for col in list_cols_dependennt_vars])], dfNumerique['Débit sève (L/j)'])
#type(dfNumerique.loc[:, ~dfNumerique.columns.isin([col.name for col in list_cols_dependennt_vars])])
#dfNumeriqueIndepVar = dfNumerique[:, ~dfNumerique.columns.isin([col.name for col in list_cols_dependennt_vars])]

In [ ]:
print("coefs: ", list_reg[0].coef_.round(2))
print("intercept: ", list_reg[0].intercept_)

fonction vérif distr normale

In [ ]:
def print_dist_norm(pCol, pXlabel, pYlabel="Fonction de densité f(x)", pNbRuns=1000):
    grille_x = np.linspace(pCol.min(), pCol.max(), pNbRuns)
    dx=(pCol.max()-(pCol.min()))/(pNbRuns-1)
    mu, sigma = sts.norm.fit(pCol.values)
    param=sts.norm.fit(pCol.values)
    pdf = sts.norm.pdf(grille_x, mu, sigma)
    ax=pCol.plot.hist(density=True, bins = 10, color = 'blue', edgecolor = 'black')
    ax.set_xlabel(pXlabel)
    ax.plot(grille_x, pdf, linewidth=3, color = 'red')
    ax.set_ylabel(pYlabel)